# Load modules

In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
from FNORUNet_5layer_model import *

import operator
from functools import reduce
from functools import partial

from timeit import default_timer
import scipy.io
import math
import gc
import glob

# Load data (will take a while) and process

In [2]:
hf_r = h5py.File(f'/scratch/groups/smbenson/shale_clean.hdf5', 'r')
data_x_shale = np.array(hf_r.get('x'))
data_SG_shale = np.array(hf_r.get('SG'))
data_P_shale = np.array(hf_r.get('P'))
data_P_init_shale = np.array(hf_r.get('P_init'))
hf_r.close()
print('Checkpoint 1')

Checkpoint 1


In [3]:
# GAS SATURATION DATA -------------------------------------------------
# Z SCORE NORMALIZATION
SG_mean = np.mean(data_SG_shale)
SG_std = np.std(data_SG_shale)
print(SG_mean)
print(SG_std)
    
data_SG_shale = (data_SG_shale - SG_mean)/(SG_std)

data_x = np.concatenate([data_x_shale], axis=0)
data_sg = np.concatenate([data_SG_shale], axis=0)

data_nr = data_x.shape[0]
test_nr = 600
train_nr = data_nr - test_nr

np.random.seed(0)
shuffle_index = np.random.choice(data_nr, data_nr, replace=False)

data_x = data_x[shuffle_index, ...]
data_sg = data_sg[shuffle_index, ...]

print('Checkpoint 2')

idx = [0,6,12,18,19,20,21,22,23]
data_x_fit = np.zeros((data_x.shape[0], len(idx)+3, 96, 200))
for j, index in enumerate(idx):
    data_x_fit[:,j,:,:] = data_x[:,index,:,:]
    
print('Checkpoint 3')
    
dz = 2.083330
dx = [0.1]

with open('DRV.txt') as f:
    for line in f:
        line = line.strip().split('*')
        dx.append(float(line[-1]))
dx = np.cumsum(dx)
grid_x = dx/np.max(dx)
grid_x = grid_x[1:]
grid_y = np.linspace(0, 200, 96)/np.max(dx)

data_x_fit[:,-3,:,:] = grid_x[np.newaxis, np.newaxis, :]
data_x_fit[:,-2,:,:] = grid_y[np.newaxis, :, np.newaxis]
data_x_fit[:,-1,:,:] = np.ones(data_x_fit[:,-1,:,:].shape)

data_x_fit[:,-3,:,:] = data_x_fit[:,-3,:,:]/np.max(data_x_fit[:,-3,:,:])
data_x_fit[:,-2,:,:] = data_x_fit[:,-2,:,:]/np.max(data_x_fit[:,-2,:,:])

x_in = data_x_fit.transpose((0,2,3,1))
SG = data_sg.transpose((0,2,3,1))

x_in = x_in.astype(np.float32)
SG = SG.astype(np.float32)

x_in = torch.from_numpy(x_in)
SG = torch.from_numpy(SG)

print('Checkpoint 4')

# a input u output
train_a = x_in[:train_nr,:,:,:]
train_u = SG[:train_nr,:,:,:]

test_a = x_in[train_nr:train_nr+ test_nr,:,:,:]
test_u = SG[train_nr:train_nr+ test_nr,:,:,:]

T = 24

train_a = train_a[:,:,:,np.newaxis,:]
test_a = test_a[:,:,:,np.newaxis,:]

train_a = train_a.repeat([1,1,1,T,1])
test_a = test_a.repeat([1,1,1,T,1])

print('Checkpoint 5')

t = np.cumsum(np.power(1.421245, range(24)))
t /= np.max(t)
for i in range(24):
    train_a[:,:,:,i,-1] = t[i]
    test_a[:,:,:,i,-1] = t[i]
    
train_a_SG = train_a
test_a_SG = test_a
train_u_SG = train_u
test_u_SG = test_u
    
print('Checkpoint 6')

0.019367479650039064
0.10341674964334545
Checkpoint 2
Checkpoint 3
Checkpoint 4
Checkpoint 5
Checkpoint 6


In [4]:
# PRESSURE BUILDUP ---------------------------------------------------
data_dP_shale = data_P_shale - data_P_init_shale

# Z-Score Normalization
dP_mean = np.mean(data_dP_shale)
dP_std = np.std(data_dP_shale)
print(dP_mean)
print(dP_std)

data_dP_shale = (data_dP_shale - dP_mean)/(dP_std)

data_x = np.concatenate([data_x_shale], axis=0)
data_dP = np.concatenate([data_dP_shale], axis=0)

data_nr = data_x.shape[0]
test_nr = 100
train_nr = data_nr - test_nr

np.random.seed(0)
shuffle_index = np.random.choice(data_nr, data_nr, replace=False)

data_x = data_x[shuffle_index, ...]
data_dP = data_dP[shuffle_index, ...]

print('Checkpoint 2')

idx = [0,6,12,18,19,20,21,22,23]
data_x_fit = np.zeros((data_x.shape[0], len(idx)+3, 96, 200))
for j, index in enumerate(idx):
    data_x_fit[:,j,:,:] = data_x[:,index,:,:]

print('Checkpoint 3')
    
dz = 2.083330
dx = [0.1]

with open('DRV.txt') as f:
    for line in f:
        line = line.strip().split('*')
        dx.append(float(line[-1]))
dx = np.cumsum(dx)
grid_x = dx/np.max(dx)
grid_x = grid_x[1:]
grid_y = np.linspace(0, 200, 96)/np.max(dx)

data_x_fit[:,-3,:,:] = grid_x[np.newaxis, np.newaxis, :]
data_x_fit[:,-2,:,:] = grid_y[np.newaxis, :, np.newaxis]
data_x_fit[:,-1,:,:] = np.ones(data_x_fit[:,-1,:,:].shape)

data_x_fit[:,-3,:,:] = data_x_fit[:,-3,:,:]/np.max(data_x_fit[:,-3,:,:])
data_x_fit[:,-2,:,:] = data_x_fit[:,-2,:,:]/np.max(data_x_fit[:,-2,:,:])

x_in = data_x_fit.transpose((0,2,3,1))
dP = data_dP.transpose((0,2,3,1))

x_in = x_in.astype(np.float32)
dP = dP.astype(np.float32)

x_in = torch.from_numpy(x_in)
dP = torch.from_numpy(dP)

print('Checkpoint 4')

# [3616, 96, 200, 12] = [num reservoirs, vertical length, lateral length, 12 parameters]
# [3616, 96, 200, 12] = [..., ..., ..., 24 time steps for prediction]

# a input u output
train_a = x_in[:train_nr,:,:,:]
train_u = dP[:train_nr,:,:,:]

test_a = x_in[train_nr:train_nr+ test_nr,:,:,:]
test_u = dP[train_nr:train_nr+ test_nr,:,:,:]

T = 24

train_a = train_a[:,:,:,np.newaxis,:]
test_a = test_a[:,:,:,np.newaxis,:]

train_a = train_a.repeat([1,1,1,T,1])
test_a = test_a.repeat([1,1,1,T,1])

print('Checkpoint 5')

t = np.cumsum(np.power(1.421245, range(24)))
t /= np.max(t)
for i in range(24):
    train_a[:,:,:,i,-1] = t[i]
    test_a[:,:,:,i,-1] = t[i]
    
train_a_dP = train_a
test_a_dP = test_a
train_u_dP = train_u
test_u_dP = test_u

print('Checkpoint 6')

1.1323901246254815
9.17773280445466
Checkpoint 2
Checkpoint 3
Checkpoint 4
Checkpoint 5
Checkpoint 6


# Load gas saturation model and predict 500 samples

In [6]:
gpu_or_cpu = 'gpu'
normMethod = "zscore"

# Set torch device
device = torch.device('cuda:0')

In [13]:
n = 500
eval_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_a_SG[:n, ...], train_u_SG[:n, ...]), batch_size=1, shuffle=False)

from FNORUNet_5layer_model import *
model1 = torch.load('/scratch/users/andchu/FNOUNet/saved_models/SG3d_FNORUNet_199ep_32width_12m1_12m2_12m3_3000train_200eps_l2err_0.0005lr_1zerr_zscorenorm_andrew_FNORUNet4_5layer', map_location=torch.device('cuda:0'))

t1 = default_timer()
with torch.no_grad():
    for x, y in eval_loader:
        x, y = x.to(device), y.to(device)
        pred = model1(x).view(-1,96,200,24)
        
t2 = default_timer()
print(t2 - t1, " seconds to predict ", n, " samples of SG model")

35.69571893301327  seconds to predict  500  samples of SG model


In [17]:
# Time (s) per prediction:
35.69571893301327 / 500

0.07139143786602653

# Load pressure buildup model and predict 500 samples

In [16]:
n = 500
eval_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_a_dP[:n, ...], train_u_dP[:n, ...]), batch_size=1, shuffle=False)

from FNORUNet_4layer_model import *
model2 = torch.load('/scratch/users/andchu/FNOUNet/saved_models/dP3d_FNORUNet_249ep_32width_12m1_12m2_12m3_3000train_250eps_l1err_0.0005lr_1zerr_zscorenorm_andrew_FNORUNet4_4layer', map_location=torch.device('cuda:0'))


t1 = default_timer()
with torch.no_grad():
    for x, y in eval_loader:
        x, y = x.to(device), y.to(device)
        pred = model2(x).view(-1,96,200,24)
        
t2 = default_timer()
print(t2 - t1, " seconds to predict ", n, " samples of dP model")

35.42577957699541  seconds to predict  500  samples of dP model


In [18]:
# Time (s) per prediction:
35.42577957699541 / 500

0.07085155915399081

In [22]:
# order of magnitude speed-up for pressure buildup model
10 * 60 / (35.42577957699541 / 500) # 10 min / prediction time
# ~ 1 x 10^4

8468.409265291435

# See number of parameters for each model

In [ ]:
# SG FINAL
#model = torch.load('/scratch/users/andchu/FNOUNet/saved_models/SG3d_FNORUNet_199ep_32width_12m1_12m2_12m3_3000train_200eps_l2err_0.0005lr_1zerr_zscorenorm_andrew_FNORUNet4_5layer', map_location=torch.device('cuda:0'))

# dP FINAL
# model = torch.load('/scratch/users/andchu/FNOUNet/saved_models/dP3d_FNORUNet_249ep_32width_12m1_12m2_12m3_3000train_250eps_l1err_0.0005lr_1zerr_zscorenorm_andrew_FNORUNet4_4layer', map_location=torch.device('cuda:0'))

# SG no derivative errors
# model = torch.load('/scratch/users/andchu/FNOUNet/saved_models/SG3d_FNORUNet_99ep_32width_12m1_12m2_12m3_3000train_100eps_l2err_0.0005lr_0zerr_zscorenorm_andrew_FNORUNet4_5layer_0rerr', map_location=torch.device('cuda:0'))

# SG only dy/dr
# model = torch.load('/scratch/users/andchu/FNOUNet/saved_models/SG3d_FNORUNet_99ep_32width_12m1_12m2_12m3_3512train_100eps_l2err_0.0005lr_0zerr_zscorenorm_andrew_FNORUNet3_5layer', map_location=torch.device('cuda:0'))

# dP no derivative errors
# model = torch.load('/scratch/users/andchu/FNOUNet/saved_models/dP3d_FNORUNet_99ep_32width_12m1_12m2_12m3_3000train_100eps_l1err_0.0005lr_0zerr_zscorenorm_andrew_FNORUNet4_4layer_0rerr', map_location=torch.device('cuda:0'))

# dP only dy/dr
model = torch.load('/scratch/users/andchu/FNOUNet/saved_models/dP3d_FNORUNet_99ep_32width_12m1_12m2_12m3_3000train_100eps_l1err_0.0005lr_0zerr_zscorenorm_andrew_FNORUNet4_4layer', map_location=torch.device('cuda:0'))

total_params = sum(
	param.numel() for param in model.parameters()
)
f'Number of model parameters: {total_params:,}'